## Animated Altitude Timeline (Dash + Plotly)

This notebook builds an **interactive animated altitude chart** for migrating birds, using:

- `pandas` for data loading and cleaning  
- `plotly.graph_objects` for the line chart  
- `dash` + `dash_bootstrap_components` for the web app and controls  

Use the **Play** button and **slider** to animate how each bird’s altitude changes over time.

In [33]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import os
import sys

# Optional: if your project has a src/ structure and you need imports from parent
sys.path.append(os.path.abspath(".."))

# Adjust this if your notebook is in a different folder
FILE_PATH = "../data/bird_migration.csv"

### 1. Load & Prepare the Data

- Read the CSV  
- Convert `date_time` to datetime  
- Clean `altitude` (fill NaNs, remove outliers)  
- Sort by time  
- Create global helper variables:
  - `GLOBAL_MIN_DATE`, `GLOBAL_MAX_DATE`
  - `ALL_BIRDS` (for the dropdown)
  - `total_seconds` (duration of the dataset for the animation)

In [34]:
try:
    df = pd.read_csv(FILE_PATH)
    df["date_time"] = pd.to_datetime(df["date_time"])
    
    # Clean altitude
    df["altitude"] = df["altitude"].fillna(0)
    
    # Remove extreme altitude outliers
    df = df[(df["altitude"] >= -100) & (df["altitude"] <= 2000)]
    
    # Sort by time
    df = df.sort_values("date_time")
    
    # Copy for safety
    df_all_data = df.copy()
    
    # Global helpers
    GLOBAL_MIN_DATE = df_all_data["date_time"].min()
    GLOBAL_MAX_DATE = df_all_data["date_time"].max()
    ALL_BIRDS = sorted(df_all_data["bird_name"].unique().tolist())
    
    print(f"Data Loaded: {len(df_all_data)} points.")
    print(f"Date range: {GLOBAL_MIN_DATE}  →  {GLOBAL_MAX_DATE}")
    print(f"Number of birds: {len(ALL_BIRDS)}")

except Exception as e:
    print(f"Critical Error while loading data: {e}")
    df_all_data = pd.DataFrame()
    ALL_BIRDS = []
    GLOBAL_MIN_DATE = pd.Timestamp.now()
    GLOBAL_MAX_DATE = pd.Timestamp.now()

Data Loaded: 60425 points.
Date range: 2013-08-15 00:01:08+00:00  →  2014-04-30 23:59:34+00:00
Number of birds: 3


### 2. Build Slider Marks for the Timeline

We drive the animation with a **0–100% slider**, but we show the **real dates** as labels.

- Compute total duration (`total_seconds`)
- Create marks at 0%, 20%, 40%, 60%, 80%, 100%
- Use `"Month Day\nYear"` formatting for readable marks

In [35]:
total_seconds = (GLOBAL_MAX_DATE - GLOBAL_MIN_DATE).total_seconds()

slider_marks = {}

if total_seconds > 0:
    for i in range(0, 101, 20):  # 0, 20, 40, 60, 80, 100
        fraction = i / 100
        date_mark = GLOBAL_MIN_DATE + pd.Timedelta(seconds=total_seconds * fraction)
        
        slider_marks[i] = {
            "label": date_mark.strftime("%b %d\n%Y"),  # e.g. "Aug 15\n2013"
            "style": {"fontSize": "12px", "whiteSpace": "pre-line"}
        }
else:
    slider_marks = {0: "Start", 100: "End"}

slider_marks

{0: {'label': 'Aug 15\n2013',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}},
 20: {'label': 'Oct 05\n2013',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}},
 40: {'label': 'Nov 26\n2013',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}},
 60: {'label': 'Jan 17\n2014',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}},
 80: {'label': 'Mar 10\n2014',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}},
 100: {'label': 'Apr 30\n2014',
  'style': {'fontSize': '12px', 'whiteSpace': 'pre-line'}}}

### 3. Function to Build the Altitude Figure

This function:

- Takes the **slider value (0–100)** and **selected birds**
- Computes the **cutoff time** based on the percentage of the date range
- Filters data up to that time
- Draws one line per bird
- Keeps the axes **fixed** so the chart doesn’t jump around during animation

In [36]:
def build_animated_average_altitude(slider_value, selected_birds):
    if df_all_data.empty or not selected_birds:
        return go.Figure().update_layout(
            title="Select birds to display data",
            template="plotly_white"
        )

    if isinstance(selected_birds, str):
        selected_birds = [selected_birds]

    # Convert slider (0-100) to a datetime cutoff
    fraction = slider_value / 100
    cutoff_time = GLOBAL_MIN_DATE + pd.Timedelta(seconds=total_seconds * fraction)

    fig = go.Figure()

    bird_colors = {
        "Eric": "#FF5733",
        "Nico": "#33FF57",
        "Sanne": "#3357FF"
    }

    # compute maximum altitude across all selected birds
    visible = df_all_data[df_all_data["bird_name"].isin(selected_birds)]
    max_alt = visible["altitude"].max() if not visible.empty else 100

    for bird in selected_birds:
        bird_df = df_all_data[df_all_data["bird_name"] == bird]
        slice_df = bird_df[bird_df["date_time"] <= cutoff_time]

        if slice_df.empty:
            continue

        # compute running average
        slice_df = slice_df.copy()
        slice_df["avg_alt"] = slice_df["altitude"].expanding().mean()

        fig.add_trace(go.Scatter(
            x=slice_df["date_time"],
            y=slice_df["avg_alt"],
            mode="lines",
            name=f"{bird} (Avg Altitude)",
            line=dict(color=bird_colors.get(bird, "#888"), width=2.0),
            opacity=0.9
        ))

    fig.update_layout(
        title=f"<b>Average Altitude Over Time — up to {cutoff_time.strftime('%Y-%m-%d')}</b>",
        xaxis=dict(
            title="Date",
            range=[GLOBAL_MIN_DATE, GLOBAL_MAX_DATE]
        ),
        yaxis=dict(
            title="Average Altitude (m)",
            range=[-20, 300]   
),
        template="plotly_white",
        height=450,
        margin=dict(l=40, r=40, t=60, b=40),
        hovermode="x unified",
        legend=dict(
            orientation="h",
            x=1, xanchor="right",
            y=1.05
        )
    )

    return fig

### 4. Create the Dash App Layout

We build a small Dash app with:

- A graph for the altitude animation  
- A dropdown to select birds  
- A Play button  
- A slider for the timeline  
- An Interval component that drives the animation

In [37]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUMEN])

app.layout = dbc.Container([
    
    html.H2("Animated Altitude Timeline", className="my-4 text-center"),
    
    dbc.Card([
        dbc.CardBody([
            dcc.Graph(id="altitude-chart", animate=False)
        ])
    ], className="shadow-sm mb-4"),
    
    dbc.Card([
        dbc.CardBody([
            html.Label("Select Birds:", className="fw-bold"),
            dcc.Dropdown(
                id="bird-selector",
                options=[{"label": b, "value": b} for b in ALL_BIRDS],
                value=[],  # start empty
                multi=True
            ),
            
            html.Hr(),
            
            dbc.Row([
                dbc.Col(
                    html.Button(
                        "Play", id="alt-play", n_clicks=0,
                        className="btn btn-success w-100"
                    ),
                    width=2
                ),
                dbc.Col(
                    dcc.Slider(
                        id="alt-slider",
                        min=0, max=100, step=0.5,
                        value=0,
                        marks=slider_marks,
                        updatemode="drag"
                    ),
                    width=10
                )
            ], align="center"),
            
            dcc.Interval(
                id="alt-interval",
                interval=100,     # 100 ms
                n_intervals=0,
                disabled=True     # start paused
            )
        ])
    ], className="bg-light")
    
], fluid=True)

### 5. Callbacks

We now add the **Dash callbacks**:

1. **Toggle Play/Pause**  
2. **Advance the slider** when playing  
3. **Update the altitude chart** when the slider or bird selection changes

In [38]:
@app.callback(
    [Output("alt-interval", "disabled"),
     Output("alt-play", "children")],
    Input("alt-play", "n_clicks"),
    State("alt-interval", "disabled")
)
def toggle_play(n_clicks, currently_disabled):
    if n_clicks:
        # Flip state: if disabled → enable and show "Pause", else disable and show "Play"
        new_state = not currently_disabled
        return new_state, ("Pause" if currently_disabled else "Play")
    # Initial state
    return True, "Play"


@app.callback(
    Output("alt-slider", "value"),
    Input("alt-interval", "n_intervals"),
    State("alt-slider", "value")
)
def advance_slider(n_intervals, current_value):
    if current_value >= 100:
        return 0
    return current_value + 0.5


@app.callback(
    Output("altitude-chart", "figure"),   # only one figure now
    [
        Input("alt-slider", "value"),
        Input("bird-selector", "value")
    ]
)
def update_altitude_chart(slider_value, selected_birds):
    return build_animated_average_altitude(slider_value, selected_birds)

### 6. Run the Server

Run the Dash app.

In VS Code / Jupyter:

- This cell will start the server.
- Open the link it prints, e.g. `http://127.0.0.1:8061/`
- Use the dropdown to select birds, then press **Play**.

In [39]:
if __name__ == "__main__":
    app.run(debug=True, port=8062)